In [0]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import cv2 as cv

import keras
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D,MaxPooling2D, Activation, Dropout, BatchNormalization, Input
from keras.models import Sequential, Model

from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [0]:
import os
import glob

In [63]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount("/content/drive")

gpg: keybox '/tmp/tmp3h2ot53u/pubring.gpg' created
gpg: /tmp/tmp3h2ot53u/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
E: Unable to locate package google-drive-ocamlfuse
Mounted at /content/drive


In [64]:
total_images = 0
major = os.listdir("drive/My Drive/training")
full_path = []
image_labels = []
all_labels = ['donuts', 'salad', 'lasagna', 'pancakes', 'pizza', 'risotto', 'cat', 'dog', 'monkey', 'squirrel', 'daisy', 'dandelion', 'sunflower', 'rose', 'tulip']

print("Loading file structure...")
for a in major:
    for b in os.listdir("drive/My Drive/training/" + a):
        full_path.append("drive/My Drive/training/" + a + "/" + b + "/")
        
print("Loading training images...")
train_images = []
for i in full_path:
    images_in_folder = 0
    label = i.split('/')[4]
    for file in glob.glob(i + "*.jpg"):
        img = cv.cvtColor(cv.imread(file),cv.COLOR_BGR2RGB)
        img = cv.resize(img, (128,128))
        total_images+=1
        train_images.append(img)
        image_labels.append(all_labels.index(label))
        images_in_folder += 1
    print("The total number of images in %s = %d" % (i,images_in_folder))
print("The total number of images in data = " + str(total_images))

Loading file structure...
Loading training images...
The total number of images in drive/My Drive/training/flowers/rose/ = 588
The total number of images in drive/My Drive/training/flowers/daisy/ = 577
The total number of images in drive/My Drive/training/flowers/dandelion/ = 789
The total number of images in drive/My Drive/training/flowers/tulip/ = 744
The total number of images in drive/My Drive/training/flowers/sunflower/ = 552
The total number of images in drive/My Drive/training/food/salad/ = 354
The total number of images in drive/My Drive/training/food/pizza/ = 362
The total number of images in drive/My Drive/training/food/lasagna/ = 348
The total number of images in drive/My Drive/training/food/risotto/ = 364
The total number of images in drive/My Drive/training/food/donuts/ = 101
The total number of images in drive/My Drive/training/food/pancakes/ = 356
The total number of images in drive/My Drive/training/animals/squirrel/ = 386
The total number of images in drive/My Drive/tr

In [0]:
X_train = np.array(train_images, np.float32) / 255.

In [66]:
image_labels = to_categorical(image_labels)
print (image_labels.shape)

(6406, 15)


In [0]:
# Normalization

mean_img = X_train.mean(axis=0)
std_dev = X_train.std(axis = 0)
X_norm = (X_train - mean_img)/ std_dev

In [0]:
from sklearn.utils import shuffle
X_norm, image_labels = shuffle(X_norm, image_labels, random_state=0)

In [69]:
Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(X_norm, image_labels, test_size=0.2, random_state=7)
print(Xtrain.shape)


(5124, 128, 128, 3)


In [73]:
model = Sequential()
model.add(BatchNormalization(input_shape=Xtrain.shape[1:]))
model.add(Conv2D(32, kernel_size=(3, 3), activation= 'relu',padding= 'same'))
model.add(Conv2D(32, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.3))

model.add(Conv2D(64, kernel_size=(3, 3), activation= 'relu', padding='same'))
model.add(Conv2D(64, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation= 'relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 42, 42, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 42, 42, 64)        18496     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 40, 40, 64)        36928     
__________

In [0]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')

In [72]:
trained_model = model.fit(Xtrain, Ytrain, epochs = 30, shuffle = True, batch_size = 100)

Epoch 1/30
5124/5124 [==============================] - 12s 2ms/step - loss: 2.4420 - acc: 0.1501
Epoch 2/30
5124/5124 [==============================] - 11s 2ms/step - loss: 2.0008 - acc: 0.2799
Epoch 3/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.7720 - acc: 0.3511
Epoch 4/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.6376 - acc: 0.4081
Epoch 5/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.5204 - acc: 0.4549
Epoch 6/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.4202 - acc: 0.4877
Epoch 7/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.3207 - acc: 0.5265
Epoch 8/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.2625 - acc: 0.5451
Epoch 9/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.1920 - acc: 0.5652
Epoch 10/30
5124/5124 [==============================] - 11s 2ms/step - loss: 1.1648 - acc: 0.5827
Epoch 11/30
5124/51

In [0]:
model.save('model.h5')

In [0]:
from google.colab import files

files.download('model.h5')